# Whole module

In [ ]:
from sklearn.utils import shuffle
import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D,MaxPooling3D, Conv3D, Flatten,Lambda, Dropout
from tensorflow.keras.models import Sequential
from keras.models import load_model
from faceRecon import FaceExtractorMultithread, FaceExtractor

route = '/home/pabloarga/Data'

print('Loading dataframes...')
fragments = []
for i in range(5):
    # Assuming each dataframe is stored as a separate table in the HDF5 file
    if i%10==0:
        print(f'{i} dataframes loaded')
    chunk = pd.read_hdf(f'{route}/dataframe{i}_FaceForensics.h5', key=f'df{i}')
    fragments.append(chunk)
#fragments = [pd.read_hdf(f'dataframes/CelebDB/dataframe{i}_600videos.h5', key=f'df{i}') for i in range(2)]#6
df = pd.concat(fragments)

print(df.describe())
print(df.dtypes)
print(df['label'].value_counts())

X = df.drop(['label'], axis = 1)
y = df['label']

print('Dividing dataset into train and test...')
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify = y)
X_train = np.stack(X_train['face'], axis=0)
X_test = np.stack(X_test['face'], axis=0)


In [ ]:
print('Creating model...')
model = Sequential()
model.add(Input(shape=(200, 200, 3)))
model.add(Lambda(lambda x: x/255.0)) #normalizamos los valores de los pixeles
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
from MetricsModule import TrainingMetrics
#prueba de que imprime las stats
metrics = TrainingMetrics(model,"/home/pabloarga/Results")
metrics.train(X_train, y_train, X_test, y_test, epochs=10)

# Test de bach training

In [ ]:
import os
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Flatten,Lambda,  Input,Dropout
from tensorflow.keras.models import Sequential
from MetricsModule import TrainingMetrics

route =  'P:\TFG\Datasets\dataframes_small' #'/home/pabloarga/Data' 
resultsPath = 'P:\TFG\Datasets\dataframes_small\\results' #'/home/pabloarga/Results'

model = Sequential()
model.add(Input(shape=(200, 200, 3)))
model.add(Lambda(lambda x: x/255.0)) #normalizamos los valores de los pixeles -> mejora la eficiencia
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))  # Dropout for regularization
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model2 = Sequential()
model2.add(Input(shape=(200, 200, 3)))
model2.add(Lambda(lambda x: x/255.0)) #normalizamos los valores de los pixeles -> mejora la eficiencia
model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Flatten())
model2.add(Dropout(0.2))  # Dropout for regularization
model2.add(Dense(512, activation='relu'))
model2.add(Dense(1, activation='softmax'))

model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model3 = Sequential()
model3.add(Input(shape=(200, 200, 3)))
model3.add(Lambda(lambda x: x/255.0)) #normalizamos los valores de los pixeles -> mejora la eficiencia
model3.add(Conv2D(64, (3, 3), activation='relu'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Conv2D(128, (3, 3), activation='relu'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Conv2D(128, (3, 3), activation='relu'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Flatten())
model3.add(Dropout(0.2))  # Dropout for regularization
model3.add(Dense(512, activation='relu'))
model3.add(Dropout(0.3))  # Dropout for regularization
model3.add(Dense(512, activation='relu'))
model3.add(Dense(1, activation='softmax'))

model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [ ]:
from MetricsModule import TrainingMetrics
import tensorflow as tf
from tensorflow.keras import layers
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Crear un nuevo modelo encima del modelo base
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

metrics = TrainingMetrics(model, resultsPath)
metrics.batches_train(route,nBatches = 1 , epochs = 2)



# Importamos el modelo y probamos con un ejemplo

In [ ]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from FaceReconModule import FaceExtractorMultithread
import pandas as pd
import numpy as np
import os
modelPath = os.path.join(resultsPath,'model.keras')

#cargamos un video y lo procesamos frame por frame
videoPath = 'P:\TFG\Datasets\FaceForensics\manipulated_sequences-fake\DeepFakeDetection\c23\\videos\\01_15__outside_talking_pan_laughing__02HILKYO.mp4'
#creamos un dataframe con el path al video y con la label 0 de fake
df = pd.DataFrame({'video': [videoPath], 'label': [0]})
faceExtractor = FaceExtractorMultithread(400) #cada 2 frames
imagesDataset = faceExtractor.transform(df)

print(imagesDataset['face'])

#Cargamos el modelo
model = load_model(modelPath,safe_mode=False)
#probamos el modelo manualmente
y_pred = model.predict(np.stack(imagesDataset['face'], axis=0))
#matriz de confusión
y_real = imagesDataset['label']
print(confusion_matrix(y_real, y_pred))


# Tests on datasets

In [1]:
from sklearn.utils import shuffle
import os
import numpy as np
from scipy import ndimage
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D,MaxPooling3D, Conv3D, Flatten,Lambda, Dropout
from tensorflow.keras.models import Sequential
from keras.models import load_model
from FaceReconModule import FaceExtractorMultithread, FaceExtractor




"""
TODO comentar
"""
def augment(row):
    image = np.array(row['face']).reshape((200, 200, 3))  # Replace height and width with the dimensions of your images

    # Perform augmentations only on fake images 
    if row['label'] == 1:
        # Flip the image
        flipped_image = np.fliplr(image)
        # Rotate the image
        rotated_image = ndimage.rotate(image, 15)  # Adjust the angle as needed

        # Add the augmented images as new examples
        new_row_flipped = row.copy()
        new_row_flipped['face'] = flipped_image.flatten().tolist()
        new_row_flipped['label'] = 1
        new_row_rotated = row.copy()
        new_row_rotated['face'] = rotated_image.flatten().tolist()
        new_row_rotated['label'] = 1

        return pd.DataFrame([row, new_row_flipped, new_row_rotated])

    return pd.DataFrame([row])





folderPath = 'C:\\Users\\pablo\\Desktop\\TFG (1)\\test_datasets\\dataframes'

nBatches = 10

numDataframes = len([name for name in os.listdir(folderPath) if os.path.isfile(os.path.join(folderPath, name))])
#Calculamos el tamaño de cada fragmento
fragmentSize = int(numDataframes/nBatches)

for i in range(nBatches):
    fragments = [pd.read_hdf(f'{folderPath}/dataframe{j}_FaceForensics.h5', key=f'df{j}') for j in range(fragmentSize*i,fragmentSize*(i+1))]
    df = pd.concat(fragments)
    #contamos el número de fakes y reales en el dataframe
    print(df['label'].value_counts())
    #print("After augmentation")
    #df = pd.concat(df.apply(augment, axis=1).tolist(), ignore_index=True)
    #print(df['label'].value_counts())
    print('-------')

ModuleNotFoundError: No module named 'FaceReconModule'

In [ ]:
from keras.models import load_model
import cv2

#cargamos el modelo del escritorio
modelPath = '/home/pabloarga/Results/2024-03-04 14.39.53/model2024-03-04 14.39.53.keras'
videoTest = '/home/pabloarga/testVideos/testVideo.mp4'

#Cargamos el modelo
model = load_model(modelPath,safe_mode=False,compile=False)
#obtenemos el número de frames del video
cap = cv2.VideoCapture(videoTest)
nFramesVideo = cap.get(cv2.CAP_PROP_FRAME_COUNT)
#Procesamos el video frame por frame
faceExtractor = FaceExtractorMultithread(nFramesVideo) #cada 2 frames
frames = faceExtractor.process_video(videoTest,1)[0]

#probamos el modelo manualmente
y_pred = model.predict(np.stack(frames, axis=0))
print(y_pred)

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Lambda, Input, Dropout, PReLU
from tensorflow.keras.models import Sequential
from MetricsModule import TrainingMetrics

# Define the constant value for PReLU alpha
value_PReLU = 0.25

# Input Layer
inputs = layers.Input(shape=(200, 200, 3))

# Function to add Convolutional layer with PReLU activation
def conv_prelu(filters, kernel_size, name):
    conv_layer = layers.Conv2D(filters, kernel_size, padding='same', name=name)
    prelu_layer = PReLU(alpha_initializer=Constant(value=value_PReLU))
    return Sequential([conv_layer, prelu_layer])

# Conv1_1 and Conv1_2 Layers
x = conv_prelu(64, (3, 3), 'conv1_1')(inputs)
x = conv_prelu(64, (3, 3), 'conv1_2')(x)
x = layers.Dropout(0.25)(x)  # Adding dropout after Conv1_2

# Pool1 Layer
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)

# Conv2_1 and Conv2_2 Layers
x = conv_prelu(128, (3, 3), 'conv2_1')(x)
x = conv_prelu(128, (3, 3), 'conv2_2')(x)
x = layers.Dropout(0.25)(x)  # Adding dropout after Conv2_2

# Pool2 Layer
pool2_output = layers.MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(x)

# Now you can use pool2_output as input for other layers
conv3_1 = conv_prelu(128, (3, 3), 'conv3_1')(pool2_output)
conv3_2 = conv_prelu(128, (3, 3), 'conv3_2')(conv3_1)
pool3 = layers.MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3_2)
conv4_1 = conv_prelu(128, (3, 3), 'conv4_1')(pool3)
conv4_2 = conv_prelu(128, (3, 3), 'conv4_2')(conv4_1)

conv5_2 = conv_prelu(128, (3, 3), 'conv5_2')(pool2_output)
conv5_3 = conv_prelu(128, (3, 3), 'conv5_3')(conv5_2)

conv5_1 = conv_prelu(128, (3, 3), 'conv5_1')(pool2_output)
concat_1 = layers.Concatenate(name="concat_1")([conv3_2, conv5_1, conv5_3])
pool5 = layers.MaxPooling2D((2, 2), strides=(2, 2), name='pool5')(concat_1)

conv6_2 = conv_prelu(128, (3, 3), 'conv6_2')(pool5)
conv6_3 = conv_prelu(128, (3, 3), 'conv6_3')(conv6_2)

conv6_1 = conv_prelu(128, (3, 3), 'conv6_1')(pool5)
concat_2 = layers.Concatenate(name="concat_2")([conv4_2, conv6_1, conv6_3])

pool4 = layers.MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(concat_2)

flatten = Flatten()(pool4)

#fc = layers.Dense(1024, name='fc')(flatten)
#fc = layers.Dropout(0.5)(fc)  # Adding dropout before the fully connected layer

#fc_class = layers.Dense(4096, name='fc_class')(fc)

# Softmax Output Layer
outputs = layers.Dense(1, activation='sigmoid', name='out')(flatten)

# Compile the model (add optimizer, loss function, etc.)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

routeServer = '/home/pabloarga/MiniData'
resultsPathServer = '/home/pabloarga/MiniData' 

metrics = TrainingMetrics(model, resultsPathServer, modelDescription = 'test')
metrics.batches_train(routeServer, nBatches=1, epochs=1)  # Divide the whole dataset into <nbatches> fragments and train <epochs> epochs with each


2024-05-12 18:56:51.706437: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.


AttributeError: 'TrainingMetrics' object has no attribute 'storeTrainingScript'

In [ ]:
# app.py
import os
import tensorflow as tf
import numpy as np
import base64
import sys
#My modules
sys.path.append("..")
from training.DataProcessing.FaceReconModule import FaceExtractorMultithread

faceExtractor = FaceExtractorMultithread() 

# Process the video
video_path = '/home/pabloarga/testVideos/fakeVideo_wwe.mp4'
videoFrames, processedFrames = faceExtractor.process_video_to_predict(video_path)


# Make predictions
predictions = model.predict(np.stack(processedFrames, axis=0))
predictions = [float(value) for value in predictions]
mean = np.mean(predictions)

{
    'predictions': {
        'data': predictions
    },
    'mean': mean,
    'nFrames': len(predictions),
}

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Dropout
from sklearn.model_selection import train_test_split

# Load the data
df = pd.read_hdf('C:\\Users\\pablo\\Desktop\\TFG (1)\\test_datasets\\sequences_100\\sequences_2.h5', key='df2')

# Extract sequences and labels
seqs = df['sequences'].to_list()
labels = df['label'].astype(np.uint8).to_numpy()

# Print information about the data
print(df.describe())
print(df.dtypes)
print(f"Number of sequences: {len(seqs)}")
print(f"Shape of the first image in the first sequence: {seqs[0][0].shape}")

# Normalize image data to [0, 1] and ensure sequences are numpy arrays
#seqs = np.array([np.array(sequence) / 255.0 for sequence in seqs])

# Ensure the sequences have the correct shape (num_sequences, num_frames, height, width, channels)
# Adjust the shape based on your image dimensions and channels (e.g., (20, 200, 200, 3) for RGB images)
# In this example, let's assume the images are 200x200 and RGB
#seqs = np.array([np.array(sequence).reshape(20, 200, 200, 3) for sequence in seqs])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(seqs, labels, test_size=0.2, random_state=42)

# Print sizes to ensure everything is correct
print(f"Training set size: {len(X_train)}")
for sequence in seqs:
    print(len(sequence))
print(f"Testing set size: {len(X_test)}")
print(f"Training labels size: {len(y_train)}")
print(f"Testing labels size: {len(y_test)}")

# Define the model
model = Sequential()

# CNN part (to process each frame)
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(20, 200, 200, 3)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))

# RNN part (to process sequences of frames)
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))

# Fully connected layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(seqs, labels, epochs=10, batch_size=4, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')


       label
count   16.0
mean     1.0
std      0.0
min      1.0
25%      1.0
50%      1.0
75%      1.0
max      1.0
sequences    object
label         int64
dtype: object
Number of sequences: 16
Shape of the first image in the first sequence: (198, 198, 3)
Training set size: 12
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
Testing set size: 4
Training labels size: 12
Testing labels size: 4


ValueError: Data cardinality is ambiguous:
  x sizes: 198, 196, 200, 200, 193, 200, 200, 197, 200, 200, 200, 190, 198, 199, 206, 202, 202, 200, 196, 188, 196, 203, 206, 205, 200, 202, 194, 195, 200, 200, 200, 192, 197, 200, 198, 200, 202, 203, 204, 198, 200, 200, 193, 200, 206, 193, 202, 197, 191, 188, 193, 193, 200, 200, 200, 200, 200, 68, 199, 210, 199, 197, 194, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 193, 198, 197, 196, 194, 198, 200, 200, 185, 200, 184, 200, 190, 174, 194, 200, 200, 200, 200, 195, 193, 202, 186, 188, 192, 190, 200, 190, 191, 198, 216, 209, 211, 221, 199, 212, 192, 200, 205, 204, 112, 198, 190, 200, 200, 199, 184, 203, 190, 200, 203, 199, 197, 200, 200, 190, 192, 199, 196, 193, 200, 200, 198, 195, 200, 195, 190, 200, 199, 195, 192, 190, 195, 200, 192, 190, 187, 188, 193, 187, 196, 200, 200, 200, 200, 200, 194, 200, 200, 201, 197, 200, 201, 200, 60, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 188, 200, 200, 200, 200, 200, 200, 200